In [1]:
import sys, os
PATH = os.path.dirname(os.path.abspath(os.curdir))
if PATH not in sys.path:
    sys.path.insert(0, PATH)

from pathos.multiprocessing import ProcessPool
import src.Simulator as sim_system
import src.Optimizer as opt
import src.SimGrad as sim_diff
import src.Grad_based_methods as grad_based
import scipy as sp
import numpy as np
import torch
import tqdm


###* Create Simulator object
reactions_file = "../reactions/reactionsSimpleV1.json"

const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

initial_state_dict = {'O_F': 0.1, 'O2_F':0.1 ,'O_S': 0.1, 'Vdb_S':0.1, 
                    'Odb_S': 0.1, 'CO_F': 0.1, 'CO2_F':0.1, 'CO_S': 0.1, 
                    'COdb_S': 0.0}

###* Functions for the data transformation
def compute_flux(const_dict, exp_dict, specie, molar_mass):
    den = exp_dict.get(specie, 0.0)
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux


def compute_remaining_flux(const_dict, exp_dict, molar_mass): 
    den = exp_dict['N'] - exp_dict['O'] - exp_dict['CO']
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux

####? EavgMB data extracted from the Booth et al. 2019 paper
p_data_exp = [0.2, 0.3, 0.4, 0.5, 0.6, 0.75, 1.5]
EavgMB_data = [1.04, 0.91, 0.87, 0.83, 0.77, 0.5, 0.001]
interpolator = sp.interpolate.interp1d(p_data_exp, EavgMB_data, kind='linear', fill_value=0.001, bounds_error=False)


transformations_exp = {
    'Tw':       lambda const_dict, exp_dict: exp_dict['Tw'] + 273.15,
    'fluxO' :   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O', 0.016),
    'fluxO2' :  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O2', 0.032),
    'fluxO3' :  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O3', 0.048),
    'fluxC':    lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'C', 0.012),
    'fluxCO':   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO', 0.028),
    'fluxCO2':  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO2', 0.048),
    'EavgMB':   lambda const_dict, exp_dict: interpolator(exp_dict['pressure']).item(),
    'Ion':      lambda const_dict, exp_dict: 1e14 * exp_dict["current"]
}

output_folder_path = "../Buffer_Data"
exp_data_file = "Experimental_data_CO_Jorge.hdf5"
exp_file = os.path.join(output_folder_path, exp_data_file)

sim = sim_system.Simulator(reactions_file, const_dict, exp_file, initial_state_dict, transformations_exp=transformations_exp)



###* Create optimization and diff objects

##! define parameters to optimize
# def func_optimization(params, flag='numpy'):
    
#     A_d, B_d, E_d, A_D, B_D, E_D, SF_1, SF_2, SF_3, SF_4 = params
    
#     if flag=='numpy':
#         nu_d_mod = lambda T: 1e15 * (B_d + A_d * np.exp(E_d/(const_dict['R'] * T)))
#         nu_D_mod = lambda T: 1e13 * (B_D + A_D * np.exp(E_D/(const_dict['R'] * T)))
#     elif flag=='torch':
#         nu_d_mod = lambda T: 1e15 * (B_d + A_d * torch.exp(E_d/(const_dict['R'] * T)))
#         nu_D_mod = lambda T: 1e13 * (B_D + A_D * torch.exp(E_D/(const_dict['R'] * T)))
#     else:
#         raise ValueError(f"{flag} does not exist")
    
#     dict_mod_vec = [
#     {"id": 2, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
#     {"id": 10, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
#     {"id": 31, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    
#     {"id": 5, "rate": None, "model_dict": {"nu_D": nu_D_mod}},
#     {"id": 7, "rate": None, "model_dict": {"nu_D": nu_D_mod}},
#     {"id": 8, "rate": None, "model_dict": {"nu_D": nu_D_mod}},
    
#     {"id": 34, "rate": None, "model_dict": {"SF": SF_1}},
#     {"id": 35, "rate": None, "model_dict": {"SF": SF_2}},
#     {"id": 36, "rate": None, "model_dict": {"SF": SF_3}},
#     {"id": 37, "rate": None, "model_dict": {"SF": SF_4}},
#     ]
#     return dict_mod_vec


def func_optimization(params, flag='numpy'):
    
    SF_1, SF_2, SF_3, SF_4, SF_5 = params
    # SF_vec = params
    
    # SF_vec = tuple(SF_vec)
    # print(SF_vec)
    # if flag=='numpy':
    #     nu_d_mod = lambda T: 1e15 * (B_d + A_d * np.exp(E_d/(const_dict['R'] * T)))
    #     nu_D_mod = lambda T: 1e13 * (B_D + A_D * np.exp(E_D/(const_dict['R'] * T)))
    # elif flag=='torch':
    #     nu_d_mod = lambda T: 1e15 * (B_d + A_d * torch.exp(E_d/(const_dict['R'] * T)))
    #     nu_D_mod = lambda T: 1e13 * (B_D + A_D * torch.exp(E_D/(const_dict['R'] * T)))
    # else:
        # raise ValueError(f"{flag} does not exist")
    
    # dict_mod_vec = [
    # {"id": 1, "rate": None, "model_dict": {"SF": SF_1}},
    # {"id": 2, "rate": None, "model_dict": {"SF": SF_2}},
    # {"id": 3, "rate": None, "model_dict": {"SF": SF_3}},
    # {"id": 4, "rate": None, "model_dict": {"SF": SF_4}},
    # {"id": 5, "rate": None, "model_dict": {"SF": SF_5}}
    # ]
    
    # dict_mod_vec = [
    # {"id": 34, "rate": None, "model_dict": {"SF": SF_1}},
    # {"id": 35, "rate": None, "model_dict": {"SF": SF_2}},
    # {"id": 36, "rate": None, "model_dict": {"SF": SF_3}},
    # {"id": 37, "rate": None, "model_dict": {"SF": SF_4}},
    # {"id": 38, "rate": None, "model_dict": {"SF": SF_5}},
    # ]
    
    
    dict_mod_vec = [
    {"id": 1, "rate": None, "model_dict": {"SF": SF_1}},
    {"id": 3, "rate": None, "model_dict": {"SF": SF_2}},
    {"id": 36, "rate": None, "model_dict": {"SF": SF_3}},
    {"id": 37, "rate": None, "model_dict": {"SF": SF_4}},
    {"id": 38, "rate": None, "model_dict": {"SF": SF_5}},
    ]
    
    return dict_mod_vec
    
    
    
    # for i in range(len(params)):
    #     dict_mod_vec[i]['model_dict']['SF'] = SF_vec[i]
    
    return dict_mod_vec
    # dict_mod_vec = [dict_mod_vec[i] for i in range(len(params))]
    
    # print(dict_mod_vec)
    # return dict_mod_vec


##! define the default parameters
# params_default = (1e-4, 1e-5, 19.0, 1e-4, 1e-5, 19.0, 1e-1, 1e-2, 1e-1, 1e-2)
# params_default = (1e-1, 1e-2, 1e-1, 1e-2)
params_default = [1.0]*5


def loss_function(exp, teo, flag='numpy'):
    
    func = ((teo-exp)**2)/(exp**2)
    if flag == 'numpy':
        return np.mean(func)
    elif flag == 'torch':
        return torch.mean(func)
    else:
        raise ValueError(f"{flag} does not exist")


optimizer = opt.Optimizer(sim, 
                        lambda params: func_optimization(params, 'numpy'), 
                        lambda exp, teo: loss_function(exp, teo, 'numpy')
                        )

diff = sim_diff.SimDiff(sim, 
                        lambda params: func_optimization(params, 'torch'),
                        params_default=params_default,
                        gamma_exp_data=sim.gamma_exp_data_arr,
                        loss_function=lambda exp, teo: loss_function(exp, teo, 'torch')
                        )


###* Optimization
def loss_and_grads(params, opt_object, diff_object):
    loss_val, frac_solutions_arr, rates_arr, _, gammas_predicted_arr = opt_object.objective_function_diff(params)
    grad_val = diff_object.objective_function_grad(params, frac_solutions_arr, rates_arr, gammas_predicted_arr)
    
    print("loss_val: ", loss_val, "params: ", params)
    return loss_val, grad_val.numpy()



def loss(params, opt_object):
    loss_val, frac_solutions_arr, rates_arr, _, gammas_predicted_arr = opt_object.objective_function_diff(params)
    grad_val = diff_object.objective_function_grad(params, frac_solutions_arr, rates_arr, gammas_predicted_arr)
    
    print("loss_val: ", loss_val, "params: ", params)
    return loss_val, grad_val.numpy()


  d[CO2_F]/dt = -CO2_F*r_29 + r_28*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_F*O_S*r_39 - CO_F*r_31 - CO_F*r_33 - 0.02*CO_F*r_35*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) + r_30*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_S]/dt = CO_F*r_35*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) - CO_S*O_F*r_38 - CO_S*r_36 + r_32*(-CO_S - O_S - Odb_S - Vdb_S + 1.0)
  d[O2_F]/dt = -O2_F*O_F*r_15 - O2_F*r_10 - O2_F*r_12 - O2_F*r_14 + r_9*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_S*O_F*r_38 - O2_F*O_F*r_15 - 2*O_F**2*r_8 - 0.02*O_F*O_S*r_7 - 0.02*O_F*Odb_S*r_27 - 0.02*O_F*Vdb_S*r_26 - O_F*r_11 - O_F*r_2 - O_F*r_4 - 0.02*O_F*r_5*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) + r_1*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_S]/dt = -CO_F*O_S*r_39 - O_F*O_S*r_7 + O_F*r_5*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) - O_S*r_16 - O_S*r_17 - O_S*r_37 - O_S*r_6 + r_3*(-CO_S - O_S - Odb_S - Vdb_S + 1.0)
  d[Odb_S]/dt = -O_F*Odb_S*r_27 + O_F*Vdb_S*r_26 - Odb_S*r_23 - Odb_S*r_24 - Odb

In [2]:

##! define the default parameters
# params_default = (1e-4, 1e-5, 19.0, 1e-4, 1e-5, 19.0, 1e-1, 1e-2, 1e-1, 1e-2)
# params_default = (1e-1, 1e-2, 1e-1, 1e-2)

print(loss_and_grads(params_default, optimizer, diff))

params_torch:  tensor([1., 1., 1., 1., 1.], dtype=torch.float64, requires_grad=True)
loss_val:  0.2223900568498911 params:  [1.0, 1.0, 1.0, 1.0, 1.0]
(0.2223900568498911, array([ 3.20650533e-01, -1.18847645e-02, -1.55949003e-04, -4.40367394e-02,
       -9.84891167e-03]))
